### Loading dataframes and packages

In [1]:
import sys, os
os.path.dirname(sys.executable)
sys.path.append('/c4/home/sermare/fusioncharacterization_gliomas/')

from fcg.common_imports import *
import importlib
from fcg import data_utils
pd.set_option('display.max_columns', None)

In [2]:
metadata = pd.read_csv('/c4/home/sermare/tables_fusion_characterization/60_patients_metadata.csv')
gary_table = pd.read_csv('/c4/home/sermare/jupyter-notebooks/Gary_patient_samples.csv', header = 0)

In [3]:
samples_ran_arriba = glob.glob('/costellolab/data1/sermare/fusion_hg38/230502024/*/*trimming_report.txt')

def extract_file_path_version(fp):
    """
    Looks for patterns like 'V3', 'V9-2', etc. in file_path.
    We'll capture them as 'v3', 'v9-2', etc. to match the style used in SF#unique.
    """
    # Basic approach: look for 'V' + digits optionally followed by '-digits' etc.
    # e.g. "P533SF12827-V9_S290_" => we want "v9"
    # If you have multiple dashes like "V9-2", you'd handle that as well.
    # Adjust this regex if you have more complicated patterns:
    m = re.search(r'V(\d+(-\d+)?)', fp, flags=re.IGNORECASE)
    if m:
        # group(1) might be e.g. "9", or "9-2", etc.
        return m.group(1)  # e.g. "v9" or "v9-2"
    else:
        return None

# Store total reads processed per file
total_reads = {}

for file in samples_ran_arriba:
    with open(file, 'r') as f:
        for line in f:
            if "Total reads processed:" in line:
                # Extract the number using regex
                match = re.search(r"Total reads processed:\s+([\d,]+)", line)
                if match:
                    reads = int(match.group(1).replace(",", ""))  # Remove commas for conversion
                    total_reads[file] = reads

# Compute the sum of all reads processed
total_reads_sum = sum(total_reads.values())

# Print results
print(f"Total reads processed across all files: {total_reads_sum}")

df = pd.DataFrame(list(total_reads.items()), columns=["file", "processed_Arriba"])
df['file']  = df['file'].str.upper()
df['file'] = [x.split('/')[-1].replace('.fastq.gz_trimming_report.txt','').split('_L')[0] for x in df['file']]

samples_ran_arriba = glob.glob('/costellolab/data2/jocostello/rna_all/*/trimmed_fastqs/*trimming_report.txt')

# Store total reads processed per file
total_reads_jocostello = {}

for file in samples_ran_arriba:
    with open(file, 'r') as f:
        for line in f:
            if "Total reads processed:" in line:
                # Extract the number using regex
                match = re.search(r"Total reads processed:\s+([\d,]+)", line)
                if match:
                    reads = int(match.group(1).replace(",", ""))  # Remove commas for conversion
                    total_reads_jocostello[file] = reads    
            elif "Processed reads:" in line:
                # Extract the number using regex
                match = re.search(r"Processed reads:\s+([\d,]+)", line)
                if match:
                    reads = int(match.group(1).replace(",", ""))  # Remove commas for conversion
                    total_reads_jocostello[file] = reads

df_jocostello = pd.DataFrame(list(total_reads_jocostello.items()), columns=["file", "processed_reads"])
df_jocostello['Parent_Folder'] = [x.split('/')[-3] for x in df_jocostello['file']]

df_jocostello['file_name']  = df_jocostello['file'].str.replace('/trimmed_fastqs', '').str.replace('_trimming_report.txt', '')
df_jocostello['full_path']  = df_jocostello['file'].str.replace('/trimmed_fastqs', '').str.replace('_trimming_report.txt', '')
df_jocostello['file_name']  = [x.split('/')[-1] for x in df_jocostello['file_name']]

df_jocostello['file']  = df_jocostello['file'].str.upper()
df_jocostello['file'] = [x.split('/')[-1].replace('.fastq.gz_trimming_report.txt','').split('_L')[0].split('_R')[0] for x in df_jocostello['file']]

df_jocostello['Patient'] = df_jocostello['file'].apply(data_utils.extract_p_number)
df_jocostello['SF#'] = df_jocostello['file'].apply(data_utils.extract_sf_number)
df_jocostello = df_jocostello[df_jocostello['Patient'].notna()]

df_jocostello['V'] = df_jocostello['file_name'].apply(extract_file_path_version)
df_jocostello = df_jocostello[df_jocostello['Patient'].notna()]
df_jocostello = df_jocostello.sort_values(by=['Parent_Folder','Patient','V'])

df_jocostello = pd.merge(left = df_jocostello, right = gary_table[['SF','Histology','Tumor']], left_on = 'SF#', right_on ='SF', how = 'left')[['Patient','Tumor','Histology','SF#','Parent_Folder','file','file_name','processed_reads','full_path']]

merged_dataframe = pd.merge(left=df_jocostello, right=df, left_on='processed_reads', right_on='processed_Arriba', how='outer')
merged_dataframe['identical_processed'] = merged_dataframe['processed_Arriba'] == merged_dataframe['processed_reads']
merged_dataframe = merged_dataframe.drop_duplicates()

# df_jocostello.to_csv('COSTELLO_RNA_Metadata.csv', index = None)

Total reads processed across all files: 100409070950


/scratch/sermare/ipykernel_2215568/1399904705.py:66: FutureWarning: The default value of regex will change from True to False in a future version.
  df_jocostello['file_name']  = df_jocostello['file'].str.replace('/trimmed_fastqs', '').str.replace('_trimming_report.txt', '')
/scratch/sermare/ipykernel_2215568/1399904705.py:67: FutureWarning: The default value of regex will change from True to False in a future version.
  df_jocostello['full_path']  = df_jocostello['file'].str.replace('/trimmed_fastqs', '').str.replace('_trimming_report.txt', '')


In [4]:
multisampled_rna = df_jocostello.groupby(['Patient','SF#'])['file'].size().reset_index().sort_values(by ='file').tail(76)

In [5]:
# Samples were not included in the analysis
df_jocostello[df_jocostello['Patient'].isin(['P413','P302','P452','P375','P340','P327','P303','P532'])].drop_duplicates('Patient')

,Patient,Tumor,Histology,SF#,Parent_Folder,file,file_name,processed_reads,full_path
23,P303,Primary,Astro,SF10809,CH04,P303SF10809V1_S87,p303SF10809v1_S87_R1_001.fastq.gz,26641455,/costellolab/data2/jocostello/rna_all/CH04/p30...
39,P375,Primary,Astro,SF11136,CH04,P375SF11136V1_S95,P375SF11136v1_S95_R2_001.fastq.gz,30283276,/costellolab/data2/jocostello/rna_all/CH04/P37...
103,P302,NaN,NaN,SF10788,CHJC05,P302SF10788V1_S1,p302SF10788v1_S1_R1_001.fastq.gz,28842166,/costellolab/data2/jocostello/rna_all/CHJC05/p...
123,P327,NaN,NaN,SF10932,CHJC05,P327SF10932V1_S22,p327SF10932v1_S22_R1_001.fastq.gz,30786841,/costellolab/data2/jocostello/rna_all/CHJC05/p...
143,P340,NaN,NaN,SF11055,CHJC05,P340SF11055V1_S32,p340SF11055v1_S32_R1_001.fastq.gz,34255351,/costellolab/data2/jocostello/rna_all/CHJC05/p...
165,P413,NaN,NaN,SF11173,CHJC05,P413SF11173V1_S11,p413SF11173v1_S11_R1_001.fastq.gz,45605532,/costellolab/data2/jocostello/rna_all/CHJC05/p...
187,P452,NaN,NaN,SF11644,CHJC06,P452SF11644V1_S27,p452SF11644v1_S27_R1_001.fastq.gz,57408319,/costellolab/data2/jocostello/rna_all/CHJC06/p...
982,P532,NaN,NaN,SF13060,CHJC17,P532SF13060-V1_S29,P532SF13060-v1_S29_R1_001.fastq.gz,44475151,/costellolab/data2/jocostello/rna_all/CHJC17/P...


In [6]:
def extract_input_fq_files(log_file):
    """
    Open a STAR Log.out file and extract the two FASTQ input filenames 
    from the line that contains '--readFilesIn'.
    """
    with open(log_file, 'r') as f:
        for line in f:
            if '--readFilesIn' in line:
                # Get the part after '--readFilesIn'
                parts = line.split('--readFilesIn', 1)
                if len(parts) > 1:
                    read_files_str = parts[1].strip()
                    tokens = read_files_str.split()
                    # Expect at least two tokens (R1 and R2)
                    if len(tokens) >= 2:
                        return tokens[0], tokens[1]
    return None, None

# List to accumulate rows for the new DataFrame
rows = []

for idx, row in metadata.iterrows():
    folder = row.get('complete_path')
    
    # Skip rows where 'complete_path' is not a valid string
    if not isinstance(folder, str):
        print(f"Skipping row {idx}: invalid complete_path {folder}")
        continue

    # Remove the last part (the filename) to get the folder
    folder = os.path.dirname(folder)
    
    # Find all Log.out files in the folder
    log_files = glob.glob(os.path.join(folder, '*Log.out'))
    if not log_files:
        print(f"No Log.out files found in folder: {folder}")
        continue

    # Assume the first found log file is the one to use
    r1_input, r2_input = extract_input_fq_files(log_files[0])
    
    # Build a new row with selected metadata and the extracted FASTQ file paths
    new_row = {
        'SF#': row['SF#'],
        'arriba_filename': row['file_path_x'],
        'patient': row['patient'],
        'Histology': row['Histology'],
        'Tumor': row['Tumor'],
        'R1': r1_input,
        'R2': r2_input
    }
    rows.append(new_row)

# Create the new DataFrame
new_df = pd.DataFrame(rows)


In [7]:
metadata_copy = metadata.drop_duplicates('file_path_x')

In [8]:
import re
import pandas as pd

def extract_version(file_path):
    match = re.search(r'V(\d+)', file_path)
    return int(match.group(1)) if match else None

# Add a new column for version numbers
metadata['version'] = metadata['file_path_x'].astype(str).apply(extract_version)

# Find the maximum version per SF#
max_versions = metadata.groupby('SF#')['version'].max().reset_index()
max_versions['version'] = max_versions['version'].fillna(0).astype(int)

# Create lookup dictionaries for file_path_x and complete_path
lookup_dict = metadata.set_index(['SF#', 'version'])['file_path_x'].to_dict()
complete_path_lookup = metadata.set_index(['SF#', 'version'])['complete_path'].to_dict()
sf_lookup_dict = metadata.set_index(['SF#'])['patient'].to_dict()

# Generate the complete version table per patient
expanded_data = [
    {
        "patient": sf_lookup_dict.get(row['SF#'], None),
        "complete_path": complete_path_lookup.get((row['SF#'], v), None),
        "SF#": row['SF#'],
        "version": int(v),
        "file_path_x": lookup_dict.get((row['SF#'], v), None)
    }
    for _, row in max_versions.iterrows()
    for v in range(1, int(row['version']) + 1)
]

# Create the final expanded DataFrame
expanded_metadata = pd.DataFrame(expanded_data)
expanded_metadata = expanded_metadata.sort_values(by=['SF#', 'version'])


In [9]:
directory_path = "/costellolab/data2/jocostello/rna_all"

# List of patients to exclude
excluded_patients = [
    'P300', 'P471', 'P457', 'P481', 'P458',
    'P499', 'P500', 'P507', 'P516', 'P498',
    'P505', 'P521', 'P524', 'P547', 'P528'
]

# Find rows where 'file_path_x' is NaN (missing)
missing_file_paths = expanded_metadata[expanded_metadata['file_path_x'].isna()]

# Remove rows for the excluded patients
missing_file_paths = missing_file_paths[~missing_file_paths['patient'].isin(excluded_patients)]

# Dictionary to store found FASTQ file paths
found_fastq_files = {}

# Iterate through each missing entry to find corresponding FASTQ files
for index, row in missing_file_paths.iterrows():
    patient = row['patient']
    sf_number = row['SF#']
    vial = row['version']
    
    # If complete_path is not NaN, remove the file name to get the folder path.
    # (If complete_path is NaN, we skip this row.)
    if pd.notna(row['complete_path']):
        folder = os.path.dirname(row['complete_path'])
    else:
        continue

    # Construct the search pattern for FASTQ files related to the sf_number and vial
    search_pattern = f"{directory_path}/*/trim*/*{sf_number}*{vial}_S*fq.gz"
    
    # Use glob to find matching files
    matching_files = glob.glob(search_pattern)
    
    # Create a name identifier from patient, SF#, and vial
    name = f"{patient}{sf_number}V{vial}"
    
    # Store the results (if no files were found, None is stored)
    found_fastq_files[name] = matching_files if matching_files else None

# # Optionally, if you want to remove these rows from the original expanded_metadata DataFrame,
# # you can do so by filtering it:
# expanded_metadata = expanded_metadata[~expanded_metadata['patient'].isin(excluded_patients) | expanded_metadata['file_path_x'].notna()]

In [10]:
expanded_metadata[~(expanded_metadata['file_path_x'].notna())]

,patient,complete_path,SF#,version,file_path_x
1,P300,None,SF10711,2,None
2,P300,None,SF10711,3,None
8,P300,None,SF10711,9,None
46,P471,None,SF11834,7,None
72,P458,None,SF11913,3,None
82,P457,None,SF11915,1,None
85,P457,None,SF11915,4,None
86,P457,None,SF11915,5,None
88,P457,None,SF11915,7,None
90,P457,None,SF11915,9,None


In [11]:
# P528 SF12790 v6 /costellolab/data2/jocostello/rna_all/CHJC26/P528SF12790-v6-1_S29_R1_001.fastq.gz
# P457 SF11915 v9 is missing /costellolab/data2/jocostello/rna_all/CHJC09/P457SF11915v9_S25_R1_001.fastq.gz

In [12]:
script_folder = "/costellolab/data1/sermare/fusion_hg38/230502024/SLURM_JOBS"
folder = "/costellolab/data1/sermare/fusion_hg38/230502024"

# Process each sample from the dictionary
for sample_name, fastq_files in found_fastq_files.items():
    print(f"Processing: {sample_name}")

    # Get the R1 and R2 file paths
    r1_file = fastq_files[0] if '_R1_' in fastq_files[0] else fastq_files[1]
    r2_file = fastq_files[1] if '_R2_' in fastq_files[1] else fastq_files[0]

    # Extract filenames from full paths
    trimmed_file_1 = os.path.basename(r1_file)
    trimmed_file_2 = os.path.basename(r2_file)

    # Use the sample name directly from the dictionary key
    sample_output = sample_name

    # Create the path for the SLURM script file
    script_file_path = os.path.join(script_folder, f"{sample_output}.sh")

    with open(script_file_path, "w") as file:
        # Write SLURM header
        file.write("#!/bin/bash\n")
        file.write(f"#SBATCH --job-name={sample_output}\n")
        file.write("#SBATCH --nodes=1\n")
        file.write("#SBATCH --partition=costellolab\n")
        file.write("#SBATCH --ntasks=1\n")
        file.write("#SBATCH --mem=50G\n")
        file.write("#SBATCH --time=99:00:00\n")
        file.write(f"#SBATCH --output={script_folder}/ERROR/%x-%j.out\n")
        file.write(f"#SBATCH --error={script_folder}/error_%j.err\n")
        file.write("#SBATCH --mail-type=END,FAIL\n")
        file.write("#SBATCH --mail-user=sergiomar10@berkeley.edu\n\n")

        # Change to the fusions directory and load required modules
        file.write(f"cd {folder}\n")
        file.write("module load CBI\n")
        file.write("module load WitteLab\n")
        file.write("module load python3\n\n")
        file.write("source /costellolab/data3/sermare/miniconda3/etc/profile.d/conda.sh\n")
        file.write("conda activate Arriba\n\n")

        # Create and move into the sample-specific output directory
        file.write(f"mkdir -p {folder}/{sample_output}\n")
        file.write(f"cd {folder}/{sample_output}\n\n")

        # Build the STAR and Arriba command pipeline
        star_command = (
            "STAR "
            "--runThreadN 8 "
            "--genomeDir /costellolab/data3/sermare/arriba_hg38/STAR_index_hg38_GENCODE38 --genomeLoad NoSharedMemory "
            f"--readFilesIn {r1_file} {r2_file} --readFilesCommand zcat "
            "--outStd BAM_Unsorted --outSAMtype BAM Unsorted --outSAMunmapped Within --outBAMcompression 0 "
            "--outFilterMultimapNmax 50 --peOverlapNbasesMin 10 --alignSplicedMateMapLminOverLmate 0.5 "
            "--alignSJstitchMismatchNmax 5 -1 5 5 "
            "--chimSegmentMin 10 --chimOutType WithinBAM HardClip --chimJunctionOverhangMin 10 --chimScoreDropMax 30 "
            "--chimScoreJunctionNonGTAG 0 --chimScoreSeparation 1 --chimSegmentReadGapMax 3 --chimMultimapNmax 50 | "
            "arriba "
            "-x /dev/stdin "
            f"-o {sample_output}_fusions.tsv -O {sample_output}_fusions.discarded.tsv "
            "-a /costellolab/data3/sermare/arriba_hg38/hg38.fa -g /costellolab/data3/sermare/arriba_hg38/GENCODE38.gtf "
            "-b /costellolab/data3/sermare/arriba_v2.3.0/database/blacklist_hg38_GRCh38_v2.3.0.tsv.gz "
            "-k /costellolab/data3/sermare/arriba_v2.3.0/database/known_fusions_hg38_GRCh38_v2.3.0.tsv.gz "
            "-t /costellolab/data3/sermare/arriba_v2.3.0/database/known_fusions_hg38_GRCh38_v2.3.0.tsv.gz "
            "-p /costellolab/data3/sermare/arriba_v2.3.0/database/protein_domains_hg38_GRCh38_v2.3.0.gff3"
        )
        file.write(star_command + "\n")

In [13]:
# P528 SF12790 v6 /costellolab/data2/jocostello/rna_all/CHJC26/P528SF12790-v6-1_S29_R1_001.fastq.gz
# P457 SF11915 v9 is missing /costellolab/data2/jocostello/rna_all/CHJC09/P457SF11915v9_S25_R1_001.fastq.gz
found_fastq_files

{}

In [14]:
found_fastq_files_other = {'P528SF12790-v6-1': ['/costellolab/data2/jocostello/rna_all/CHJC26/P528SF12790-v6-1_S29_R1_001.fastq.gz',
 '/costellolab/data2/jocostello/rna_all/CHJC26/P528SF12790-v6-1_S29_R2_001.fastq.gz'], 
                    'P528SF12790-v6-2' : ['/costellolab/data2/jocostello/rna_all/CHJC26/P528SF12790-v6-2_S30_R1_001.fastq.gz',
 '/costellolab/data2/jocostello/rna_all/CHJC26/P528SF12790-v6-2_S30_R2_001.fastq.gz'], 
                    'P457SF11915v9':['/costellolab/data2/jocostello/rna_all/CHJC09/P457SF11915v9_S25_R1_001.fastq.gz',
 '/costellolab/data2/jocostello/rna_all/CHJC09/P457SF11915v9_S25_R2_001.fastq.gz']}

In [15]:
script_folder = "/costellolab/data1/sermare/fusion_hg38/230502024/SLURM_JOBS"
folder = "/costellolab/data1/sermare/fusion_hg38/230502024"

# Process each sample from the dictionary
for sample_name, fastq_files in found_fastq_files_other.items():
    print(f"Processing: {sample_name}")

    # Get the R1 and R2 file paths
    r1_file = fastq_files[0] if '_R1_' in fastq_files[0] else fastq_files[1]
    r2_file = fastq_files[1] if '_R2_' in fastq_files[1] else fastq_files[0]

    # Extract filenames from full paths
    trimmed_file_1 = os.path.basename(r1_file)
    trimmed_file_2 = os.path.basename(r2_file)

    # Use the sample name directly from the dictionary key
    sample_output = sample_name

    # Create the path for the SLURM script file
    script_file_path = os.path.join(script_folder, f"{sample_output}.sh")

    with open(script_file_path, "w") as file:
        # Write SLURM header
        file.write("#!/bin/bash\n")
        file.write(f"#SBATCH --job-name={sample_output}\n")
        file.write("#SBATCH --nodes=1\n")
        file.write("#SBATCH --partition=costellolab\n")
        file.write("#SBATCH --ntasks=1\n")
        file.write("#SBATCH --mem=50G\n")
        file.write("#SBATCH --time=99:00:00\n")
        file.write(f"#SBATCH --output={script_folder}/ERROR/%x-%j.out\n")
        file.write(f"#SBATCH --error={script_folder}/error_%j.err\n")
        file.write("#SBATCH --mail-type=END,FAIL\n")
        file.write("#SBATCH --mail-user=sergiomar10@berkeley.edu\n\n")

        # Change to the fusions directory and load required modules
        file.write(f"cd {folder}\n")
        file.write("module load CBI\n")
        file.write("module load WitteLab\n")
        file.write("module load python3\n\n")
        file.write("source /costellolab/data3/sermare/miniconda3/etc/profile.d/conda.sh\n")
        file.write("conda activate Arriba\n\n")

        # Create and move into the sample-specific output directory
        file.write(f"mkdir -p {folder}/{sample_output}\n")
        file.write(f"cd {folder}/{sample_output}\n\n")

        # Build the STAR and Arriba command pipeline
        star_command = (
            "STAR "
            "--runThreadN 8 "
            "--genomeDir /costellolab/data3/sermare/arriba_hg38/STAR_index_hg38_GENCODE38 --genomeLoad NoSharedMemory "
            f"--readFilesIn {r1_file} {r2_file} --readFilesCommand zcat "
            "--outStd BAM_Unsorted --outSAMtype BAM Unsorted --outSAMunmapped Within --outBAMcompression 0 "
            "--outFilterMultimapNmax 50 --peOverlapNbasesMin 10 --alignSplicedMateMapLminOverLmate 0.5 "
            "--alignSJstitchMismatchNmax 5 -1 5 5 "
            "--chimSegmentMin 10 --chimOutType WithinBAM HardClip --chimJunctionOverhangMin 10 --chimScoreDropMax 30 "
            "--chimScoreJunctionNonGTAG 0 --chimScoreSeparation 1 --chimSegmentReadGapMax 3 --chimMultimapNmax 50 | "
            "arriba "
            "-x /dev/stdin "
            f"-o {sample_output}_fusions.tsv -O {sample_output}_fusions.discarded.tsv "
            "-a /costellolab/data3/sermare/arriba_hg38/hg38.fa -g /costellolab/data3/sermare/arriba_hg38/GENCODE38.gtf "
            "-b /costellolab/data3/sermare/arriba_v2.3.0/database/blacklist_hg38_GRCh38_v2.3.0.tsv.gz "
            "-k /costellolab/data3/sermare/arriba_v2.3.0/database/known_fusions_hg38_GRCh38_v2.3.0.tsv.gz "
            "-t /costellolab/data3/sermare/arriba_v2.3.0/database/known_fusions_hg38_GRCh38_v2.3.0.tsv.gz "
            "-p /costellolab/data3/sermare/arriba_v2.3.0/database/protein_domains_hg38_GRCh38_v2.3.0.gff3"
        )
        file.write(star_command + "\n")

Processing: P528SF12790-v6-1
Processing: P528SF12790-v6-2
Processing: P457SF11915v9
